## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette, PneumaticSampleLoader
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('../..')
import stober_synthesis_utils as stober

In [3]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2025_03_20_TEOSgradient.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [4]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
response in connect:  [True, True, True, True]


In [5]:
sample_table = pd.read_csv('2025_03_20_mesoporous_TEOSgradient - Sheet1.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [6]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [7]:
sample_table['well'] = ['A1', 'A2', 'A3', 'A4', 'B1']

In [8]:
sample_table

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_voume,well
0,1,371,733,1420,2590,2190,2700,A1
1,2,311,733,1441,2629,2190,2700,A2
2,3,251,733,1462,2668,2190,2700,A3
3,4,191,733,1483,2707,2190,2700,A4
4,5,131,733,1504,2746,2190,2700,B1


In [143]:
jubilee.park_tool()

gcode cmd:  M114
gcode cmd:  T-1


In [9]:
sample_table['water_volume'].sum()

7310

## Load Tools

In [95]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")
syringe_1_4 = syringe.HTTPSyringe.from_config(0, '../../../science-jubilee/src/science_jubilee/tools/configs/1cc_4_syringe.json')

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}
Syringe name:  1cc_4
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_4'}


In [96]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)
jubilee.load_tool(syringe_1_4)

In [97]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3
surfactant_syringe = syringe_1_4

## Load water syringe with water

In [98]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [105]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [106]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 150)

In [107]:
water_syringe.set_pulsewidth(water_syringe.full_position+250, s = 500)

In [108]:
water_syringe.load_syringe(590, water_syringe.full_position+250)

Loaded syringe, remaining volume 590 uL


## Load ammonia syringe

In [109]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [110]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [119]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [120]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 200)

In [121]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+250, s = 500)

In [122]:
ammonia_syringe.load_syringe(560, ammonia_syringe.full_position+250)

Loaded syringe, remaining volume 560 uL


## Load TEOS syringe

In [123]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [128]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [129]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 200)

In [130]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+300, s = 200)

In [131]:
teos_syringe.load_syringe(510, teos_syringe.full_position+300)

Loaded syringe, remaining volume 510 uL


## load ethanol syringe

In [132]:
#jubilee.park_tool()

In [133]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [134]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


## Don't need to load surfactant syringe

In [135]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load AFL

## Load labware


In [136]:
samples = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples.manual_offset([(31.1, 169.0), (121.1, 169.0), (121.1, 113.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [137]:
stocks_main = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_main.manual_offset([(30.5, 266.0),  (120.5, 266.0), (120.5, 210.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [138]:
teos_stocks = [stocks_main[0]]
ammonia_stocks = [stocks_main[1]]
water_stocks = [stocks_main[2]]
ethanol_stocks = [stocks_main[3]]
ctab_stocks = [stocks_main[4]]
f127_stocks = [stocks_main[5]]
rinse_stocks_precursor = [stocks_main[6], stocks_main[7], stocks_main[8]]
rinse_stocks_teos = [stocks_main[9], stocks_main[10], stocks_main[11]]

## Check sample alignment

In [47]:
jubilee.pickup_tool(mix_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

169.0


Is the tip centered over sample A1? 


169.0


Is the tip centered over sample A2? 


169.0


...

Is the tip centered over sample B4? 


113.0


Is the tip centered over sample C1? 


113.0


Is the tip centered over sample C2? 


In [49]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0,82c085e1-a523-40ce-9e89-4fa0bf546525,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B2
1,1,747534b1-7370-49a5-bdc1-4fce5227f391,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B3
2,2,e5f6e82f-0282-46c8-be77-d8914adca9b5,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B4
3,3,c87ac840-ddde-43ab-a8e3-b5e3eb5a273d,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C1
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C2


In [57]:
jubilee.park_tool()


In [106]:
loc = samples['A8']
jubilee.move_to(x = loc.x, y = loc.y)
print(loc.y)
jubilee.move_to(z = loc.top_+7)

175.8614659806774


# Experiment


In [139]:
location_lookup = {row['uuid']:samples[row['well']] for i, row in sample_table.iterrows()}


In [140]:
sample_table['ethanol_volume'].sum()

13340

## add ethanol

In [47]:
def reactant_transfer(syringe, source, destination, volume, volume_buffer, rinse_stocks = None, rinse_vol = 500, n_rinse = 1, dwell_time = 3):
    # zero out syringe
    syringe.set_pulsewidth(syringe.empty_position - 1)
    syringe.load_syringe(0, syringe.empty_position-1)

    jubilee.pickup_tool(syringe)

    
    # need to account for dispenses > syringe volume
    n_dispenses = int(np.ceil(volume / (syringe.capacity - volume_buffer)))
    step_volume = volume/n_dispenses
    print(f'breaking dispense into {n_dispenses} of volume {step_volume}')
    
    for i in range(n_dispenses):
        # make sure syringe has enough volume
        if i == 0:
            syringe.aspirate(step_volume + volume_buffer, source.bottom(+5), s = 10, dwell_before = 5)
        else:
            syringe.aspirate(step_volume, source.bottom(+5), s = 10, dwell_before = 5)

    
            

        syringe.dispense(step_volume, destination.bottom(+5), s = 200)
        time.sleep(dwell_time)
        logger.info(f'Dispensed {volume} uL from {source} into {destination}')

    syringe.dispense(volume_buffer, source.bottom(+5), s = 20)

    if rinse_stocks is not None:
        print(f'Rinsing {syringe.name}') 
        for stock in rinse_stocks:
            syringe.mix(rinse_vol, n_rinse, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 1000)

        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks}')

    jubilee.park_tool()
    syringe.set_pulsewidth(syringe.empty_position - 1)
    syringe.load_syringe(0, syringe.empty_position-1)


In [46]:
surfactant_syringe.name

'1cc_4'

In [158]:
ind = 5

well = location_lookup[ind+1]
etoh_vol = sample_table.iloc[ind-1]['ethanol_volume']
f127_volume = sample_table.iloc[ind-1]['f127_voume']
ctab_volume = sample_table.iloc[ind-1]['ctab_volume']
sample_table_select = sample_table.iloc[ind:ind+1]
location_lookup_selected = {ind+1:well}

reaction_time = 60*5

KeyError: 6

In [159]:
sample_table_select

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_voume,well
4,5,131,733,1504,2746,2190,2700,B1


In [155]:
location_lookup_selected

{5: Well B1 form 20mLscintillation 12 Well Plate 18000 µL on slot 2}

In [156]:
well

Well B1 form 20mLscintillation 12 Well Plate 18000 µL on slot 2

In [157]:

start_time = time.time()

#add ethanol
reactant_transfer(mix_syringe, ethanol_stocks[0], well, etoh_vol, 500)

#add water
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup, 'water_volume', water_stocks, dwell_time = 7)
print(f'added water')
#add ammonia
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup, 'ammonia_volume', ammonia_stocks, dwell_time = 10)
print('added ammonia')
# add F127

reactant_transfer(surfactant_syringe, f127_stocks[0], well, f127_volume, 50)
print(f'added {f127_volume} f127')
reactant_transfer(surfactant_syringe, ctab_stocks[0], well, ctab_volume, 50)
print(f'added {ctab_volume} uL CTAB')
# mix precursors

stober.first_mix(jubilee, mix_syringe, 5000, location_lookup_selected, rinse_stocks_precursor, 3, n_rinse = 1)
print('mixed samples')
# add teos
teos_add_time = stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1, return_time = True)

print(f'Time from start to TEOS addition: {teos_add_time - start_time} s')

time.sleep(reaction_time - (time.time() - teos_add_time))

print('########## Complete ############')






Loaded syringe, remaining volume 0 uL
breaking dispense into 1 of volume 2707.0
Loaded syringe, remaining volume 0 uL
breaking dispense into 2 of volume 752.0
remaining vol:  949.0
remaining vol:  949.0
added water
breaking dispense into 1 of volume 733.0
remaining vol:  949.0
added ammonia
Loaded syringe, remaining volume 0 uL
breaking dispense into 3 of volume 900.0
Loaded syringe, remaining volume 0 uL
added 2700 f127
Loaded syringe, remaining volume 0 uL
breaking dispense into 3 of volume 730.0
Loaded syringe, remaining volume 0 uL
added 2190 uL CTAB
mixed samples
breaking dispense into 1 of volume 131.0
remaining vol:  949.0
Time from start to TEOS addition: 773.8089802265167 s
########## Complete ############


In [83]:
teos_add_time = stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1, return_time = True)

print(f'Time from start to TEOS addition: {teos_add_time - start_time} s')

time.sleep(reaction_time - (time.time() - teos_add_time))

print('########## Complete ############')

breaking dispense into 1 of volume 311.0
Time from start to TEOS addition: 920.250785112381 s
########## Complete ############


In [86]:
logger.info(f'Added TEOS to sample {ind} at {time.strftime("%H:%M", time.localtime(teos_add_time))}')


In [87]:
(time.time() - teos_add_time)/60

13.084396016597747

15.333333333333334

In [79]:
teos_syringe.remaining_volume

510

In [94]:
jubilee.park_tool()

In [ ]:
print(f'total time: {(time.time() - start_time)/60}')

In [193]:
psl.load_sample(mix_syringe, well, 1000)



mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

print(f'Time to load: {(time.time() - start_time)/60}')
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock, s_aspirate = 2000, s_dispense = 1000)

jubilee.park_tool()

end_time = time.time()

print(f'Total sample time: {(end_time - start_time)/60}')

status r code 200
status:  b'["State: LOADED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","[250318-01:25:43] Load stopped at voltage mean = 2.159246788024902 and stdev = 1.3360104511043107"]\n'
status r code 200
status:  b'["State: LOADED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","[250318-01:25:43] Load stopped at voltage mean = 2.159246788024902 and stdev = 1.3360104511043107"]\n'
status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rin

In [181]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock)

Loaded syringe, remaining volume 0 uL


In [132]:
sample_table

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_voume,well
0,1,371,733,1420,2590,2190,2700,A1
1,2,371,733,1420,2590,2190,2700,A2
2,3,371,733,1420,2590,2190,2700,A3
3,4,371,733,1420,2590,2190,2700,A4
4,5,371,733,1420,2590,2190,2700,B1


In [134]:
etoh_vol

2590

In [139]:
time.time()

1742352112.1766489

In [138]:
time.time() + 60*5 

1742352325.196246

In [98]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_select, location_lookup, 'ethanol_volume', ethanol_stocks, stocks_usable_volume=15000, dwell_time = 10)

breaking dispense into 1 of volume 2590.0


In [99]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

In [100]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Dispense water


In [101]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

breaking dispense into 2 of volume 710.0
remaining vol:  949.0
remaining vol:  949.0


## dispense Ammonia

In [102]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup, 'ammonia_volume', ammonia_stocks, dwell_time = 10)

breaking dispense into 1 of volume 733.0
remaining vol:  949.0


## dispense F127

In [103]:
stober.add_reactants_batch(jubilee, surfactant_syringe, mix_syringe, sample_table_select, location_lookup, 'f127_voume', f127_stocks, refill_dwell = 10)

breaking dispense into 3 of volume 900.0
remaining vol:  949
remaining vol:  949.0
remaining vol:  949.0


In [104]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)
jubilee.pickup_tool(surfactant_syringe)
for stock in rinse_stocks_precursor:
    surfactant_syringe.mix(900, 1, stock)
jubilee.park_tool()

Loaded syringe, remaining volume 0 uL


## Dispense CTAB

In [105]:
stober.add_reactants_batch(jubilee, surfactant_syringe, mix_syringe, sample_table_select, location_lookup, 'ctab_volume', ctab_stocks, refill_dwell = 10)

breaking dispense into 3 of volume 730.0
remaining vol:  949.0
remaining vol:  949.0
remaining vol:  949.0


In [106]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)
jubilee.pickup_tool(surfactant_syringe)
for stock in rinse_stocks_precursor:
    surfactant_syringe.mix(900, 1, stock)
jubilee.park_tool()

Loaded syringe, remaining volume 0 uL


## mix precursors

In [113]:
ind = 2
location_lookup_selected = {ind:location_lookup[ind]}

In [114]:
location_lookup_selected

{2: Well A2 form 20mLscintillation 12 Well Plate 18000 µL on slot 2}

In [115]:
stober.first_mix(jubilee, mix_syringe, 5000, location_lookup_selected, rinse_stocks_precursor, 3, n_rinse = 1)

## Add TEOS and mix

In [116]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1)

breaking dispense into 1 of volume 371.0
remaining vol:  949.0


## Load cell

In [125]:
psl.load_sample(mix_syringe, samples['A2'], 1000)

status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'
status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'
status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'
status r code 200
status:  b'["State: RE

In [127]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock)
jubilee.park_tool()

Loaded syringe, remaining volume 0 uL
